# NLP for Social Good: Toxic Comment Classification

### Project Overview
This project applies **Natural Language Processing (NLP)** and machine learning to address the critical issue of online toxicity and harassment. It develops a robust classification model capable of automatically identifying and flagging toxic comments on social media platforms. By providing an automated solution, this system can assist human moderators in creating a safer and more inclusive online environment.

### Dataset
The model is trained on a public dataset from the **Jigsaw Toxic Comment Classification Challenge** on Kaggle. This dataset contains a large number of social media comments with labels for various types of toxicity, including `toxic`, `severe_toxic`, `obscene`, `threat`, `insult`, and `identity_hate`. This rich, real-world dataset makes the project highly relevant and impactful.

### Methodology
1.  **Data Preprocessing and Cleaning:** The raw text data is cleaned by removing punctuation, stop words, and special characters. The text is then tokenized and converted into a numerical format suitable for a neural network.
2.  **Model Architecture:** The project uses a Bidirectional Long Short-Term Memory (**Bi-LSTM**) network. This advanced recurrent neural network can process text sequentially, learning contextual relationships from both forward and backward directions, which is crucial for understanding the nuances of human language.
3.  **Training:** The Bi-LSTM model is trained to classify comments into one or more of the toxicity categories. The training process focuses on optimizing the model's ability to accurately identify toxic content while minimizing false positives.
4.  **Evaluation:** Model performance is evaluated using key metrics that are essential for imbalanced datasets, such as the **F1-score**, **Precision**, and **Recall**. These metrics provide a more comprehensive view of the model's effectiveness than simple accuracy alone.

### Concluded Results
The Bi-LSTM model successfully identifies different types of toxic comments with a high F1-score, showcasing its ability to handle a complex, multi-label classification problem. This project demonstrates proficiency in modern NLP techniques, ethical AI application, and the ability to build a practical system to combat a real-world social problem.

### Technologies Used
- Python
- TensorFlow / Keras
- Pandas
- NumPy
- Matplotlib
- NLTK
- Jupyter Notebook

In [ ]:
# Project 12: Toxic Comment Classification

# --- Section 1: Setup and Data Loading ---
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# IMPORTANT: You will need to download the dataset from Kaggle and upload it to your Colab session.
# The file is 'train.csv' from the Jigsaw Toxic Comment Classification Challenge.
print("Loading dataset...")
try:
    # Replace 'path/to/train.csv' with the actual path in your session or Google Drive
    df = pd.read_csv('train.csv')
except FileNotFoundError:
    print("Error: 'train.csv' not found. Please upload the dataset.")
    # Example of a small synthetic dataset for demonstration if needed
    data = {'comment_text': ['This is a great movie.', 'You are an idiot.', 'I love this!', 'Go kill yourself.', 'What a terrible thing to say.'],
            'toxic': [0, 1, 0, 1, 1],
            'severe_toxic': [0, 0, 0, 1, 0],
            'obscene': [0, 0, 0, 0, 0],
            'threat': [0, 0, 0, 1, 0],
            'insult': [0, 1, 0, 0, 1],
            'identity_hate': [0, 0, 0, 0, 0]}
    df = pd.DataFrame(data)

print("Data head:")
print(df.head())

labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# --- Section 2: Data Preprocessing ---
print("\nPreprocessing text data...")

def clean_text(text):
    text = text.lower()
    text = re.sub('[{}]'.format(re.escape(string.punctuation)), '', text)
    return text

df['comment_text'] = df['comment_text'].apply(clean_text)

X = df['comment_text']
y = df[labels]

max_words = 20000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Split data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)

# --- Section 3: Building the Bi-LSTM Model ---
print("\nBuilding the Bidirectional LSTM model...")

model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(len(labels), activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# --- Section 4: Training and Evaluation ---
print("\nTraining the model...")
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

print("\nEvaluating the model...")
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

print("Classification Report (on Test Data):")
print(classification_report(y_test, y_pred, target_names=labels))

print("\nTest Accuracy:")
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Overall Test Accuracy: {accuracy*100:.2f}%")